# LTSFLinearForecaster pre-training example

This notebook demonstrates how to pre-train the ``LTSFLinearForecaster`` on panel data before fitting it to a single series. Pre-training allows the model to learn shared temporal patterns from related series, which can help when only a handful of observations are available for the final forecasting task.

In [ ]:
from sktime.datasets import load_hierarchical_sales_toydata
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.ltsf import LTSFLinearForecaster

# Load a panel dataset with multiple related product-level series
y_panel = load_hierarchical_sales_toydata()
y_panel.head()

In [ ]:
forecaster = LTSFLinearForecaster(
    seq_len=24,
    pred_len=6,
    num_epochs=3,  # keep epochs small so the example trains quickly
    batch_size=16,
    lr=1e-3,
)

# Pre-train on the full panel data
forecaster.pretrain(y_panel)
forecaster.state, forecaster.n_pretrain_instances_

In [ ]:
# Select a single series (Hobs in the Food preparation product line)
y_target = y_panel.xs(("Food preparation", "Hobs"))["Sales"]

fh = ForecastingHorizon(list(range(1, forecaster.pred_len + 1)), is_relative=True)

forecaster.fit(y_target, fh=fh)
y_pred = forecaster.predict()
print(y_pred)

In [ ]:
# Inspect which attributes were populated during pre-training
sorted(forecaster.get_pretrained_params().keys())[:5]